# Uncovering the Liars in the Political Scene of America

In [ ]:
# Imports Mimmi


In [ ]:
# Imports Martin


In [ ]:
# Imports Ben
import pandas as pd

## Loading phase

In [ ]:
DATA_DIR = 'data/'
SCHEMA = ['statement_id', 'label', 'statement', 'subject', 
          'speaker', 'profession', 'state', 'party', 
          'barely_true_counts', 'false_count', 'half_true', 
          'mostly_true', 'pants_on_fire', 'context']

# Load the datasets into pandas dataframes
test = pd.read_csv(DATA_DIR + 'test.tsv', delimiter='\t', header=None, names=SCHEMA, index_col=False)
train = pd.read_csv(DATA_DIR + 'train.tsv', delimiter='\t', header=None, names=SCHEMA, index_col=False)
valid = pd.read_csv(DATA_DIR + 'valid.tsv', delimiter='\t', header=None, names=SCHEMA, index_col=False)

## Mimmi's code

## Martin's code

## Ben's code